In [1]:
import tensorflow as tf

In [8]:
tf.reset_default_graph()

A = tf.placeholder(tf.float32, shape=(None, 3))
B = A + 5

with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A:[[1,2,3]]})
    B_val_2 = B.eval(feed_dict={A:[[1,2,3],[1,2,3]]})
    
print(B_val_1)
print(B_val_2)

[[6. 7. 8.]]
[[6. 7. 8.]
 [6. 7. 8.]]


In [37]:
import numpy as np
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()
m, n = housing.data.shape
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

n_epochs = 100
learning_rate = 0.01
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

## Mini-batch Gradient Descent

In [32]:
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  
    indices = np.random.randint(m, size=batch_size) 
    X_batch = scaled_housing_data_plus_bias[indices] 
    y_batch = housing.target.reshape(-1, 1)[indices] 
    return X_batch, y_batch

In [38]:
tf.reset_default_graph()
X = tf.placeholder(tf.float32, shape=(None, n+1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate) #### OPTIMIZER
training_op = optimizer.minimize(mse)


init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
    best_theta = theta.eval()
    saver.save(sess, "/Users/daniel/Desktop/my_model_final.ckpt")
    
best_theta

array([[ 2.0544672 ],
       [ 0.8297011 ],
       [ 0.10713524],
       [-0.3107482 ],
       [ 0.2456451 ],
       [-0.00222412],
       [-0.01061389],
       [-0.89121586],
       [-0.87524796]], dtype=float32)

In [40]:
with tf.Session() as sess:
    saver.restore(sess, "/Users/daniel/Desktop/my_model_final.ckpt")
    best_theta_restored = theta.eval()
    
best_theta_restored

INFO:tensorflow:Restoring parameters from /Users/daniel/Desktop/my_model_final.ckpt


array([[ 2.0544672 ],
       [ 0.8297011 ],
       [ 0.10713524],
       [-0.3107482 ],
       [ 0.2456451 ],
       [-0.00222412],
       [-0.01061389],
       [-0.89121586],
       [-0.87524796]], dtype=float32)